# Assignment 1

# Perform Text Classification on the data. 

The tweets have been pulled from Twitter related to coronavirus and manual tagging has been done then.

The names and usernames have been given codes to avoid any privacy concerns.

You have to only use "OriginalTweet" and "sentiment" column for the assignment. You can either drop or ignore other columns for this assignment.

# 1) Import required libraries - 2 Mark

In [1]:
#Imports
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
# for integer encoding using sklearn
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import seaborn as sns
from matplotlib import style
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
from tkinter import *
from tkinter.font import Font
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import scikitplot as skplt
from sklearn.preprocessing import scale
from pyclustertend import hopkins
from sklearn.decomposition import PCA
from tqdm import tqdm 
from math import sqrt 
from time import gmtime, strftime
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import os
##print(os.listdir("../input"))
sns.set()
from sklearn import preprocessing as pp
from sklearn.cluster import KMeans
import random
import pylab as pl
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import DBSCAN,AgglomerativeClustering, KMeans
from apyori import apriori
######## Capture the start time to check the runt ime of the whole notebook #########
startTime = strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import unicodedata
import nltk
from nltk import word_tokenize
import re  #regular expression
from bs4 import BeautifulSoup
from gensim.parsing.preprocessing import remove_stopwords
import gensim.corpora as corpora
from pprint import pprint
import gensim
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
sw = stopwords.words('english')
stemmer = SnowballStemmer("english")
url_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'


C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 2) Read dataset and perfom Text processing for the tweets ( Remove Stop words , special characters and convert the text to lowercase ) - 3 Mark

In [2]:
#Extract and read text files

tweets = pd.DataFrame()


try:
    tweets = pd.read_csv("Corona_NLP_train.csv",encoding="mbcs")
    #tweets.columns = ['OriginalTweet', 'Sentiment']
    tweets = tweets.drop(['UserName', 'ScreenName','Location','TweetAt'], axis = 1)

except Exception as e:
    print('Exception occured ',str(e))
    
    
print(tweets)

                                           OriginalTweet           Sentiment
0      @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...             Neutral
1      advice Talk to your neighbours family to excha...            Positive
2      Coronavirus Australia: Woolworths to give elde...            Positive
3      My food stock is not the only one which is emp...            Positive
4      Me, ready to go at supermarket during the #COV...  Extremely Negative
...                                                  ...                 ...
41152  Airline pilots offering to stock supermarket s...             Neutral
41153  Response to complaint not provided citing COVI...  Extremely Negative
41154  You know itÂ’s getting tough when @KameronWild...            Positive
41155  Is it wrong that the smell of hand sanitizer i...             Neutral
41156  @TartiiCat Well new/used Rift S are going for ...            Negative

[41157 rows x 2 columns]


In [3]:
#Sentence curation functions
def stopwords(text):
    '''a function for removing the stopword'''
    
    # removing the stop words and lowercasing the selected words
    #text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    text = remove_stopwords(text)
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    #return(text)
    return " ".join(text)

def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with no space, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)

def stemming(text):    
    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)     

def remove_html(text):
    '''
    remove the HTML tags and URLS from the tweets
    '''
    if text:
        # BeautifulSoup on content
        soup = BeautifulSoup(text, "html.parser")
        # Stripping all <code> tags with their content if any
        if soup.code:
            soup.code.decompose()
        # Get all the text out of the html
        text =  soup.get_text()
        # Returning text stripping out all uris
        return re.sub(url_re, "", text)
    else:
        return ""
    
def remove_emojis(text):
    
    emoji_pattern = re.compile("["
        r"\U0001F600-\U0001F64F"  # emoticons
        r"\U0001F300-\U0001F5FF"  # symbols & pictographs
        r"\U0001F680-\U0001F6FF"  # transport & map symbols
        r"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)# no emoji 
  
    return text

def remove_specialChars(text):
    text = re.sub('[^a-zA-Z0-9.\d\s]', '', text)
    return text

<>:58: DeprecationWarning: invalid escape sequence \d
<>:58: DeprecationWarning: invalid escape sequence \d
<>:58: DeprecationWarning: invalid escape sequence \d
<ipython-input-3-dc4875ae85da>:58: DeprecationWarning: invalid escape sequence \d
  text = re.sub('[^a-zA-Z0-9.\d\s]', '', text)


In [4]:
## Remove unwanted words ##

tweets_duplicate = tweets.copy()
tweets['OriginalTweet'] = tweets['OriginalTweet'].apply(remove_html)
tweets['OriginalTweet'] = tweets['OriginalTweet'].apply(remove_emojis)
tweets['OriginalTweet'] = tweets['OriginalTweet'].apply(remove_punctuation)
tweets['OriginalTweet'] = tweets['OriginalTweet'].apply(stopwords)
tweets['OriginalTweet'] = tweets['OriginalTweet'].apply(remove_specialChars)
tweets_duplicate = tweets.copy()


print(tweets)

                                           OriginalTweet           Sentiment
0                            menyrbie philgahan chrisitv             Neutral
1      advice talk neighbours family exchange phone n...            Positive
2      coronavirus australia woolworths elderly disab...            Positive
3      food stock please dont panic enough food every...            Positive
4      ready supermarket covid19 outbreak im paranoid...  Extremely Negative
...                                                  ...                 ...
41152  airline pilots offering stock supermarket shel...             Neutral
41153  response complaint provided citing covid19 rel...  Extremely Negative
41154  know its getting tough kameronwilds rationing ...            Positive
41155  wrong smell hand sanitizer starting turn coron...             Neutral
41156  tartiicat well newused rift going 70000 amazon...            Negative

[41157 rows x 2 columns]


# 3) Using the train_test_split function of Sklearn, Split train and test dataset - 1 Mark

In [5]:
# Splitting the dataset into the Training set and Test set

#Label Encode Y column 

label_encoder = LabelEncoder()
# Encode labels in column 'Sentiment'.
tweets['Sentiment']= label_encoder.fit_transform(tweets['Sentiment'])

# seperate X,Z and y variables
y = tweets['Sentiment']
X = tweets['OriginalTweet']

X=np.array(X)
y=np.array(y)
#X.shape
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

print(y_train)

[4 3 2 ... 2 2 1]


# 4) Create pipeline and define parameters for GridSearch ( You might Refer the code below ) - 1 Mark

In [6]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

text_clf

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

# 5) Perform classification (using GridSearch) - 3 Mark

In [7]:
grid_search = GridSearchCV(text_clf, tuned_parameters, cv=5, verbose=1,n_jobs=-1)
result = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.3min finished
C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:844: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


# 6) Print the confusion matrix, accuracy, F1 score on the test dataset - 2 Mark

In [8]:

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import scikitplot as skplt

#Predict the response for test dataset
y_pred = result.predict(X_test)
#y_pred_prob = grid_clf_acc.predict_proba(X_test)

confusion_matrix = confusion_matrix(y_test, y_pred)
acc_score = accuracy_score(y_test, y_pred)


#print('Best Hyperparameters: %s' % result.best_params_)
print("Confusion matrix \n",confusion_matrix)
print ("accuracy score = ",round(acc_score*100,2),"%")


#print classification report 
print('classification report is as below: \n', classification_report(y_test, y_pred))



Confusion matrix 
 [[ 336    9 1012   21  265]
 [   3  479  164   32 1320]
 [ 113   46 1622  127 1067]
 [  17   40  526  723  984]
 [  17  206  718  134 2367]]
accuracy score =  44.76 %
classification report is as below: 
               precision    recall  f1-score   support

           0       0.69      0.20      0.32      1643
           1       0.61      0.24      0.34      1998
           2       0.40      0.55      0.46      2975
           3       0.70      0.32      0.43      2290
           4       0.39      0.69      0.50      3442

    accuracy                           0.45     12348
   macro avg       0.56      0.40      0.41     12348
weighted avg       0.53      0.45      0.43     12348



In [9]:
endTime = strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())
print ("Run started at : ",startTime)
print ("Run ended at : ",endTime)

Run started at :  Sun, 11 Jul 2021 17:06:45 +0000
Run ended at :  Sun, 11 Jul 2021 17:08:11 +0000
